# Advanced Machine Learning - programming assignment 1

*Due: Friday December 2nd*

* Carboni Leonardo (0279048)
* Bais Giacomo (5355583)

### Further instructions:
* Make sure your code is properly commented.
* Submit your code in Blackboard using **one** of your accounts; we will put the grade in Blackboard for the other team member as well.
* **Make sure to name the submitted file according to your and your collaborators last name.** (submitter_collaborator.ipynb)

## Multi-armed Bandits

In this programming assignment, we will look at how we can solve a k-armed bandit problem as discussed in the lecture. Expect for winning at the slot machines, you are expect to better understand the tradeoff between exploration and exploiation. 

Here are the objectives of this assignment:
1.   Get familier with the Open-AI gym environment,
2.   Implement your own k-armed bandit environment based on the gym framework,
3.   Use an epsilon-greedy algorithm to find the optimal action for this k-armed bandit problem,
4.   Play with the parameter epsilon and identify a reasonable setting for balancing exploration and exploiation. 
    

### 1. Let's start with the OpenAI gym

Gym (https://gym.openai.com/) is a wide-used toolkit for developing and comparing reinforcement learning algorithms. 

1. Gym makes no assumptions about the structure of your agent, and is compatible with any numerical computation library, such as TensorFlow or Theano. 

2. The gym library is a collection of test problems — **environments** — that you can use to work out your reinforcement learning algorithms. These environments have a shared interface, allowing you to write general algorithms.

First, we download & install the gym library. 

In [ ]:
#!pip install gym

**Great!** Now let's import the gym class and work on a basic example of gym code.


In [ ]:
import gym

Like mentioned above, gym's main purpose is to provide a large collection of **environments** that expose a common interface. You can find a listing of those environments below (they are Markov decision process(MDP) enviroments and we will discuss MDP in our lecture), as follows:

In [ ]:
from gym import envs
print(envs.registry)

We are now going to explain how the RL framework of gym works. 
- An **ENVIRONMENT**, 
- You also have an **AGENT**,
- The agent takes an **ACTION**, in our case, 10 actions are possible to take,
- When a single **ACTION** is chosen and fed to our **ENVIRONMENT**, the **ENVIRONMENT** measures how good the action was taken and produces a **REWARD**, which is usually a numeric value.

In MDP problems, the **ENVIRONMENT** will also provides an **OBSERVATION**, which represets the state of the **ENVIRONMENT** at the current moment. In the multi-armed bandit problems, there is no **OBSERVATION** (or state). You may understand this better after the lecture about Markov decision process (MDP). 

Please read the 'Getting Started with gym' https://gym.openai.com/docs/ for better understanding the framework. 

### 2. Implement your own environment

Next, we are going to implement our own multi-arm bandit environment following the framework of gym. This enviroment is a gambiling room with ten different slot machines (a 10-armed bandit problem). Similar with examples given in the lecture, the reward of each slot machine follows a normal distribution, but the average reward (mean) and variance of each action are different. Your goal is to determine the optimal action from all possible actions/machines. 

The core gym interface is **Env**, which is the unified environment interface. There is no interface for agents. The following are the Env methods you should know:

- `step(self, action)`: Steps the environment by one timestep. Returns observation, reward, done, info.
- `reset(self)`: Resets the environment to an initial state. Returns an initial observation. Each call of `reset()` should yield an environment suitable for a new episode, independent of previous episodes. Because there is no state transition in multi-armed bandit problems, this function is not used here.
- `render(self, mode='human')`: Renders one frame of the environment. The default mode will do something human friendly, such as pop up a window. In this assignment, there is no need to create a pop up window. 

Before writing your own codes, read through the readme of github page of gym https://github.com/openai/gym. You are also recommended to read at least the codes for one simple environment and one example agent.

#### 2.1 Self-defined Slot Machine

**Please fill in the missing codes in the function sample (1 point).**

In [ ]:
import numpy as np

class SlotMachine:
    """
        A slot machine contains a reward distribution that randomly generated with restricted mean and standard deviation. 
            sample function: generates a reward at each time step based on the given reward distribition
    """
    def __init__(self):
        self.mu = np.random.uniform(-5, 5)  # mean
        self.sigma = np.random.uniform(0.5, 1)  # standard deviation

    def sample(self):
        ########## TODO: to be filled. ########## 
        return np.random.normal(loc = self.mu, scale = self.sigma)

#### 2.2 Game Environment
**Please fill in the missing codes in function step (1 point) in the environment.** 

In [ ]:
from gym import spaces

# The environment has to inherit the interface of gym.Env


class GamblingRoom(gym.Env):
    """
    A k-armed bandit environment: a gambling room with slot machines, allows the agents to interact with it.
        r_machines: A list of slot machines, each GamblingRoom contains k number of SlotMachines
    """

    def __init__(self, k):
        # initialize reward distribution for each action/machine
        self.r_machines = []
        for i in range(k):
            # each gamblingRoom contains k number of SlotMachines
            self.r_machines.append(SlotMachine())

        self.num_arms = k
        self.action_space = spaces.Discrete(self.num_arms)
        self.observation_space = spaces.Discrete(1)
        # for our bandit environment, the state is constant
        self.state = 0
        self.seed()

    # step up the environment based on the selected action,
    # return the constant state, reward, done = false, and info
    # for now, we do not have to worry about the DONE and INFO variables.
    def step(self, action):
        assert self.action_space.contains(action)
        done = False
        info = {}

        ########## TODO: to be filled. ##########
        reward = self.r_machines[action].sample()
        return self.state, reward, done, info

    def seed(self, seed=None):
        pass

    def reset(self):
        pass

    def render(self, mode='human', close=False):
        for sm in self.r_machines:
            print((sm.mu, sm.sigma))

    def close(self):
        pass


### 3. Implement an agent with the epsilon greedy algorithm

In this part, you are expected to implement an RL agent. To decide the action to take at each time step, this agent uses the epsilon greedy algorithm introduced in the lecture.

**Please fill in the missing codes in function select_action (1.5 points) and update_parameters (1 point) in the agent.** Feel free to import the needed packages if there are any.

In [ ]:
class EplisonGreedyAgent:
    def __init__(self, k, e):
        # set up the number of arms/actions
        self.num_arms = k
        # set up the value of epsilon
        self.epsilon = e
        # init the estimated values of all actions
        self.Qvalues = np.zeros(k)
        # init the numbers of time step that every action is selected
        self.stepSize = np.zeros(k)

    ##
    # select the action to take at the current time step
    # (for MDP, choose the action based on state; for k-armed bandit, no state given)
    # return: the action to take
    ##
    def select_action(self):
        ########## TODO: to be filled. ##########
        if np.random.uniform() < self.epsilon: # epsilon chance to randomly explore
            new_action = np.random.randint(0, self.num_arms)
            return new_action
        new_action = np.random.choice(np.where(self.Qvalues == self.Qvalues.max())[0])
        return new_action

    ##
    # Update the Q-values of the agent based on received rewards
    # input: action_index = the action, reward = the reward from this action
    # return: null
    ##
    def update_parameters(self, action, reward):
        ########## TODO: to be filled. ##########  
        self.stepSize[action] += 1
        self.Qvalues[action] = self.Qvalues[action] + (reward - self.Qvalues[action])/(self.stepSize[action])
        

### 4. Run the simulation, play with parameters and analyse results

Finally, we write codes for running the simulation. 

In order to decrease the effect of randomness, we usually conduct multiple simulation runs and average the results. In the implementation, you may start with one run, then use the variable `num_runs` for running multiple simulations.

In each run, you shall setup the `epsilon` and number of time step `num_episodes` (0.01 and 500 by default). Then, after the initlization of our agent and environment, **please fill in the missing codes (with ??? or TODO: to be filled). (2.5 points)**

In [ ]:
num_action = 10
num_seed = 5
num_runs = 100  # number of simulation runs
num_episodes = 500  # number of steps in each run
epsilon = 0.01

# set up the random seed
np.random.seed(num_seed)

# init the environment
env = GamblingRoom(num_action)

# delete the wrap
env = env.unwrapped

# show the action space
sims = np.zeros(num_runs)
# run multiple simulations
for i_run in range(num_runs):
    ########## TODO: to be filled. ########## 

    # init the epsilon-greedy RL agent 
    agent = EplisonGreedyAgent(num_action, epsilon)
    # in each simulation run, loop the action selection
    # save the result variables you need
    reward_tot = 0
    for _ in range(num_episodes):
        action = agent.select_action()
        reward = env.step(action)[1]
        agent.update_parameters(action, reward)
        reward_tot += reward
    sims[i_run] = reward_tot
env.close()

print(np.mean(sims))

Now it's time to examine the performance of algorithms with different epsilon values (different exploration strategies) in multiple simulation runs. 

You shall play with the parameter epsilon under 2 or 3 different gambling environments (by initlizing different reward distributions for machines). **For each environment, try at least 2 different values of epsilon and identify a reasonable epsilon value that could balance the exploration and exploiation**. Instead of handing in your codes for this part, please select one environment you have tested and describe your environment and experimental settings **(1 point)**. Then, provide an explanation on how you identify the good epsilon value in this environment and why it is a good one **(1 point)**. 

Few instructions:
- Your answer shall include two plots presenting compariable measures of the different epsilon settings (e.g. the average reward per step and % of optimal action). **(1 point)** 
- You shall present the average results from at least 100 simulation runs. Remember that the gambling environment CANNOT be changed over those runs used for calculating the average results. 
- You may adjust the total time steps when the learning needs more time for a cerain epsilon value, but do not over spend your time on this.    

**Fill in your answer (at most 300 words) with accompanying plots here.** 

You are almost done! Before handing in, make sure that the codes you hand in work, and that all plots are shown. **Submit just one file per team.** Please make sure that you submit a .zip file with images.

Again, make your you name this file according to your last names.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
num_action = 10
num_seed = 42
num_runs = 200  # number of simulation runs
num_episodes = 500  # number of steps in each run

# set up the random seed
np.random.seed(num_seed)

# init the environment
env = GamblingRoom(num_action)

# delete the wrap
env = env.unwrapped

best_action = None
best_mu = -100
for i, sm in enumerate(env.r_machines):
    if best_mu < sm.mu:
        best_mu = sm.mu
        best_action = i

steps_list = []
best_action_epsilon = []

for i_eps, epsilon in enumerate(np.arange(0, 0.11, 0.05)):
    step_ba = np.zeros(num_episodes)
    steps_reward = np.zeros((num_runs, num_episodes))
    sims = np.zeros(num_runs)
    for i_run in range(num_runs):
        agent = EplisonGreedyAgent(num_action, epsilon)
        reward_tot = 0
        for i_step in range(num_episodes):
            action = agent.select_action()
            reward = env.step(action)[1]
            agent.update_parameters(action, reward)
            reward_tot += reward
            steps_reward[i_run, i_step] = reward
            if action == best_action:
                step_ba[i_step] += 1
        sims[i_run] = reward_tot
    steps_avg_reward = np.mean(steps_reward, axis=0)
    steps_list.append(steps_avg_reward)
    best_action_epsilon.append(step_ba * 100 / num_runs)

env.close()

# average reward plot
plot1, = plt.plot(np.arange(1, 501, 1), np.array(
    steps_list[0]), label='eps = 0', linewidth=0.5)

plot2, = plt.plot(np.arange(1, 501, 1), np.array(
    steps_list[1]), label='eps = 0.05', linewidth=0.5)

plot3, = plt.plot(np.arange(1, 501, 1), np.array(
    steps_list[2]), label='eps = 0.1', linewidth=0.5)
plt.legend(handles=[plot1, plot2, plot3])
plt.xlabel("Steps")
plt.ylabel("Simulation avg reward")
plt.title("Average rewards for each step across multiple simulations")
plt.show()

# optimal action plot
plot1, = plt.plot(np.arange(1, 501, 1), np.array(
    best_action_epsilon[0]), label='eps = 0', linewidth=0.5)

plot2, = plt.plot(np.arange(1, 501, 1), np.array(
    best_action_epsilon[1]), label='eps = 0.05', linewidth=0.5)

plot3, = plt.plot(np.arange(1, 501, 1), np.array(
    best_action_epsilon[2]), label='eps = 0.1', linewidth=0.5)
plt.legend(handles=[plot1, plot2, plot3])
plt.xlabel("Steps")
plt.ylabel("Optimal action choice %")
plt.title("Optimal action percentage for each step")
plt.show()


# Conclusions
The environment chosen for the experiment is a gambling room with 10 slot machines. Each agent goes through 200 runs, and each run is composed of 500 episodes. After extensive exploration with different environments, it seemed that the best alpha values in terms of average step reward are always in the range between 0 and 0.1. Thus, for this specific environment we chose to experiment with three different alpha values: 0, 0.05 and 0.1. To select the best alpha value among these three, the average step reward and the percentage of optimal choice per step across all simulations were plotted for each alpha value. Results are shown as images in the zip file. 
As we can see in the average reward plot, at first, the greedy strategy improved a little bit quicker than the other ways, but it subsequently plateaued at a lower level. In contrast to the best feasible reward per step on this testbed of approximately 3.22, it only managed to achieve a reward of roughly 2.57. Because it frequently gets stuck completing ineffective tasks, the greedy technique consistently performs much worse over time. We can also see that the results using an epsilon of 0.05 are slightly higher than using 0.1 as the epsilon value, making it the optimal value in this case.
The optimal action graph demonstrates that only about 40% of the jobs were successfully completed using the greedy strategy. It never went back to the optimal action in the remaining section because its initial samples of it were unsatisfying. The ε-greedy approaches eventually outperform the other ways because they keep exploring and increase their odds of finding the best course of action. The epsilon = 0.1 method got a maximum percentage of optimal action choice of 94.5%, 0.5% more than the ε = 0.05 method. In this case, as said earlier, both the non-greedy methods obtained very similar results.